## First true test: can a simple R-D system for a self-limiting actication domain?

Define a function that gives the logic of the PDE system

In [ ]:
from pde import ScalarField, FieldCollection, PDEBase, CartesianGrid
import numpy as np

def make_1d_grid(length=3000, dx=10, periodic=True):
    N = int(length / dx)
    grid = CartesianGrid([[0, length]], shape=(N,), periodic=periodic)
    return grid
    
class TuringPDE1D(PDEBase):
    def __init__(self,
                 da=1.85,
                 dr=15,
                 k_a=1.11e-4,
                 k_r=0.61e-4,
                 V_A=1.0,
                 V_R=1e-2,
                 lambda_=0.01, # Lefty inhibition rate
                 n=2, m=2, p=2,
                 K_A=100.0,
                 K_R=100.0,
                 K_P=100.0,
                 bc="auto_periodic_neumann",
                 amplitude=10,
                 radius=10.0,
                 r_init="constant",
                 r_value=0.0,
                 r_noise_range=(0.2, 0.6)):
        """
        1D activator–repressor PDE system with Hill-like activation and subtractive inhibition.

        Parameters
        ----------
        da : float
            Diffusion coefficient of the activator A.
        lambda_ : float
            Scaling factor for subtractive repression by R.
        k_a : float
            Decay rate of A.
        k_r : float
            Decay rate of R.
        n, m, p : int
            Hill exponents for A activation, R inhibition of A, and A activation of R.
        K_A, K_R, K_P : float
            Hill thresholds for activation and inhibition functions.
        bc : str
            Boundary condition.
        amplitude : float
            Peak value of the initial A hotspot.
        radius : float
            Std dev of Gaussian for initial A hotspot.
        r_init : str
            Initial state of R: "constant" or "random".
        r_value : float
            Constant value of R if r_init="constant".
        r_noise_range : tuple
            Min/max range for uniform noise if r_init="random".
        """
        super().__init__()
        self.da = da
        self.dr = dr
        self.lambda_ = lambda_
        self.V_A = V_A
        self.V_R = V_R
        self.k_a = k_a
        self.k_r = k_r
        self.n = n
        self.m = m
        self.p = p
        self.K_A = K_A
        self.K_R = K_R
        self.K_P = K_P
        self.bc = bc
        self.amplitude = amplitude
        self.radius = radius
        self.r_init = r_init
        self.r_value = r_value
        self.r_noise_range = r_noise_range

    def evolution_rate(self, state, t=0):
        A, R = state

        # Hill-like terms
        A_act = self.V_A * (A**self.n) / (self.K_A**self.n + A**self.n)
        A_rep = self.lambda_ * A * (R**self.m) / (self.K_R**self.m + R**self.m)
        R_act = self.V_R * (A**self.p) / (self.K_P**self.p + A**self.p)

        # PDEs
        f_A = A_act - self.k_a * A - A_rep
        f_R = R_act - self.k_r * R

        dA_dt = self.da * A.laplace(self.bc) + f_A
        dR_dt = self.dr * R.laplace(self.bc) + f_R

        return FieldCollection([dA_dt, dR_dt])

    def get_state(self, grid):
        A = self.make_center_hotspot(grid).copy(label="Activator")
        R = self.initialize_repressor(grid).copy(label="Repressor")
        return FieldCollection([A, R])

    def make_center_hotspot(self, grid):
        field = ScalarField(grid, data=0.0)
        x = grid.axes_coords[0]
        x0 = 0.5 * (x[0] + x[-1])
        profile = self.amplitude * np.exp(-((x - x0)**2) / (2 * self.radius**2))
        field.data = profile
        return field

    def initialize_repressor(self, grid):
        if self.r_init == "constant":
            return ScalarField(grid, data=self.r_value)
        elif self.r_init == "random":
            noise = np.random.uniform(*self.r_noise_range, size=grid.shape)
            return ScalarField(grid, data=noise)
        else:
            raise ValueError(f"Unrecognized r_init method: {self.r_init}")


In [ ]:
from pde.trackers.base import TrackerBase

class NodalROITracker(TrackerBase):
    def __init__(self, grid, roi_width=500, interval=10):
        """
        Tracks Nodal distribution metrics over time in a 1D simulation.

        Parameters
        ----------
        grid : pde.CartesianGrid
            The 1D spatial grid used in the simulation.
        roi_width : float
            Width (in μm) of the region of interest centered in the domain.
        interval : int
            Number of steps between metric evaluations.
        """
        super().__init__()
        self.interval = interval
        self.grid = grid
        self.roi_width = roi_width

        self.x = grid.axes_coords[0]
        self.dx = self.x[1] - self.x[0]
        self.N_total_init = None
        self.N_roi_init = None

        x_center = 0.5 * (self.x[0] + self.x[-1])
        self.roi_mask = np.abs(self.x - x_center) <= (roi_width / 2)

        self.fraction_in_roi = []
        self.fold_change_in_roi = []
        self.fold_change_in_max = []
        self.times = []
        self.counter = 0

    def handle(self, state, time):
        if self.counter % self.interval == 0:
            A = state[0].data
            N_total = A.sum() * self.dx
            N_max = A.max()
            N_roi = A[self.roi_mask].sum() * self.dx

            if self.N_total_init is None:
                self.N_total_init = N_total
                self.N_roi_init = N_roi
                self.N_max_init = N_max

            frac_in_roi = N_roi / N_total if N_total > 0 else 0.0
            fold_change = N_roi / self.N_roi_init if self.N_roi_init > 0 else 0.0
            max_change = N_max / self.N_max_init if self.N_roi_init > 0 else 0.0

            self.fraction_in_roi.append(frac_in_roi)
            self.fold_change_in_roi.append(fold_change)
            self.fold_change_in_max.append(max_change)
            self.times.append(time)

        self.counter += 1
        
    def get_metrics(self):
        if not self.fraction_in_roi:
            return {
                "roi_fraction_final": None,
                "roi_fold_change_final": None,
                "max_fold_change_final": None
            }
    
        return {
            "roi_fraction_final": self.fraction_in_roi[-1],
            "roi_fold_change_final": self.fold_change_in_roi[-1],
            "max_fold_change_final": self.fold_change_in_max[-1]
        }

In [ ]:
from pde import CartesianGrid, PlotTracker#, SphericalGrid
# from turing_model import TuringPDE  # adjust if class is defined inline
import matplotlib.pyplot as plt

dx = 10
T = 10*3600
L = 2500
# make grid for simulation
grid = make_1d_grid(length=L, dx=dx)

# initialize model
model = TuringPDE1D(amplitude=101)

# get time step for solver
dt = 0.5 * dx**2 / model.dr

state = model.get_state(grid)
# tracker = PlotTracker(interval=100*dt)
tracker = NodalROITracker(grid, roi_width=500, interval=50)

result = model.solve(state, t_range=T, dt=dt, tracker=["progress", tracker])

# # Create initial state
# state = model.get_state(grid)


# T = 1e5
# # Set up visualization tracker
# tracker = PlotTracker(interval=100*dt)

# # Run the simulation
# result = model.solve(state, t_range=T, dt=dt, tracker=["progress", tracker])

# # Optional: display final state
# state = model.get_state(grid)
# state.plot()
# plt.suptitle("Final State of Activator–Repressor System")
# plt.show()

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook' 

fig = px.scatter(x=tracker.fraction_in_roi, y=tracker.fold_change_in_max, color=tracker.times)
fig.show(renderer="notebook")

# print(tracker.times)
# print(tracker.fraction_in_roi)
# print(tracker.fold_change_in_roi)

In [ ]:
tracker.get_metrics()

In [ ]:
tracker.results